<a href="https://colab.research.google.com/github/hedyfernando/Data_Science_Learn/blob/main/h8dsft_P0W3_hedy_fernando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Name : Hedy Fernando
## Batch : 001-HCK


### Objective :
- Mampu memperoleh data menggunakan BigQuery
- Mampu melakukan pemrosesan data sebelum melakukan perhitungan dan analisa
- Mampu menerapkan konsep statistics pada persoalan
- Memahami dasar Pengetahuan Bisnis

### Problem Statement
- Divisi Marketing dari The Look e-commerce ingin mengetahui, berapa persentase pengguna loyal yang diklasifikasikan telah melakukan minimal 3 transaksi dari Januari 2022 hingga Juni 2022.
- Berapa rata-rata,minimal dan maksimal dari transaksi yang telah dilakukan oleh user loyal.
- Berapa rata-rata,minimal dan maksimal uang yang telah dihabiskan oleh user loyal.
- Traffic Source mana yang banyak digunakan oleh pengguna loyal untuk mengakses platform The Look Ecommerce.
- Apakah variabel Traffic Source mempengaruhi variabel Loyal dan menentukan pengguna loyal atau tidak loyal.

### Working Area

- Connect to BigQuery

In [351]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = 'halo-project-358808'
client = bigquery.Client(project=project_id)

Authenticated


- Import Library

In [352]:
import pandas as pd
from scipy import stats

- Data Exploration

# 1

In [353]:
retention = client.query('''
SELECT (SELECT count(kategori) as loyal_customer from
          (SELECT user_id as users, count(order_id) as total, 
          case when count(order_id)>=3 then 'loyal' else 'not loyal' end AS Kategori
          FROM `bigquery-public-data.thelook_ecommerce.order_items`
          WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
          GROUP BY(user_id) 
          HAVING kategori='loyal'))/(SELECT count(kategori) as loyal_customer from
          (SELECT user_id as users, count(order_id) as total, 
          case when count(order_id)>=3 then 'loyal' else 'not loyal' end AS Kategori
          FROM `bigquery-public-data.thelook_ecommerce.order_items`
          WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
          GROUP BY user_id)) * 100 AS loyal_percentage
''').to_dataframe()
retention

,loyal_percentage
0,19.225251


Setelah dilakukan penarikan data dari tabel orders di database The Look Ecommerce, untuk mengklasifikasikan user loyal, yang di tentukan berdasarkan transaksi yang dilakukan minimal 3 kali oleh users. Setelah di hitung persentase dengan rumus "user loyal / keseluruhan user x 100" didapatkan hasil persentase dari user loyal sejumlah 19.225251 selama periode dari Januari - Juni 2022 dari total keseluruhan user yang ada. Bisa disimpulkan user loyal sangatlah sedikit dari keseluruhan total user yang ada, hanya sekitar 19 persen user yang melakukan transaksi lebih dari 3 kali di platform The look Ecommerce. Hal ini bisa ditinjau lagi, apakah sedikitnya jumlah user yang melakukan transaksi disebabkan oleh harga yang terlalu mahal dibanding platform lain atau bisa disebabkan barang yang di jual kurang menarik untuk pembeli di negara tertentu.(Data di analisis pada tanggal 28 Agustus 2022)

# 2

In [354]:
ct_loyal = client.query('''
SELECT AVG(total) as rata_rata, min(total) as minimal, max(total) as maximal 
from (SELECT user_id as users, count(order_id) as total,
  case when count(order_id)>=3 then 'loyal' else 'not loyal' end AS Kategori
  FROM `bigquery-public-data.thelook_ecommerce.order_items`
  WHERE created_at>="2022-01-01" and created_at<="2022-06-30"
  GROUP BY user_id
  HAVING kategori='loyal')
''').to_dataframe()
ct_loyal

,rata_rata,minimal,maximal
0,3.791723,3,11


Rata-rata transaksi yang dilakukan oleh user loyal sebanyak 3.791723 kali dari keseluruhan user loyal yang ada. Minimal transaksi yang dilakukan user loyal sebanyak 3 transaksi dan maksimal transasksi yang pernah dilakukan user sebanyak 11. (Data di analisis pada tanggal 28 Agustus 2022)

# 3

In [355]:
ct_money_loyal = client.query('''
SELECT AVG(price) AS rata_rata_spend_uang, 
min(price) AS minimal_spend_uang, 
max(price) AS maksimal_spend_uang FROM
  (SELECT user_id as users, count(order_id) as total, sum(sale_price) as price,
    case when count(order_id)>=3 then 'loyal' else 'not loyal' end AS Kategori
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at>="2022-01-01" and created_at<="2022-06-30" 
    and status!='Cancelled' and status!='Returned'
    GROUP BY user_id
    HAVING kategori='loyal')
''').to_dataframe()
ct_money_loyal

,rata_rata_spend_uang,minimal_spend_uang,maksimal_spend_uang
0,218.646915,24.21,1380.720001


Setelah melakukan perhitungan rata-rata, minimal dan maksimal pada data user loyal. Kita dapatkan bahwa user loyal rata-rata menghabiskan uang sejumlah 218.646915 dollar, untuk minimal uang yang dihabiskan dari user loyal berjumlah 24.21 dollar dan uang maksimal atau terbanyak yang pernah dihabiskan user loyal sejumlah 1380.720001 dollar. Hasil dari perhitungan tidak termasuk transaksi yang Cancelled ataupun barang yang Returned. (Data di analisis pada tanggal 28 Agustus 2022)

# 4

In [356]:
traffic_loyal = client.query('''
SELECT traffic_sources,count(users) AS users_loyal FROM
(SELECT u.traffic_source AS traffic_sources, oi.user_id AS users,
    case when count(oi.order_id)>=3 then 'loyal' else 'not loyal' end AS Kategori
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
    WHERE oi.created_at>="2022-01-01" and oi.created_at<="2022-06-30"
    GROUP BY u.traffic_source, oi.user_id
    HAVING kategori='loyal') GROUP BY traffic_sources
''').to_dataframe()
traffic_loyal

,traffic_sources,users_loyal
0,Search,4206
1,Display,223
2,Organic,865
3,Email,263
4,Facebook,339


Setelah kita hitung jumlah user loyal berdasarkan jenis traffic source yang paling banyak digunakan yaitu:
1. Search diakses sebanyak 4206
2. Organic diakses sebanyak 865
3. Facebook diakses sebanyak 339
4. Email diakses sebanyak 263
5. Display diakses sebanyak 223

dapat disimpulkan bahwa traffic source dengan tipe Search merupakan yang terbanyak mendapatkan user loyal untuk mengakses platform The Look Ecommerce. Dengan informasi ini, pihak Marketing atau Digital Marketing bisa memaksimalkan pemasaran melalui 2 tipe traffic source yang paling banyak digunakan yaitu Search melalui iklan seperti Google Adwords dan Organic menggunakan SEO. Sehingga cost iklan bisa dialihkan ke traffic source yang sangat berpengaruh atau cost iklan bisa menjadi lebih murah dengan mengetahui traffic source yang memberikan hasil maksimal.(Data di analisis pada tanggal 28 Agustus 2022)

# 5

In [357]:
connection = client.query('''
SELECT u.traffic_source AS Traffic_Sources,
    case when count(oi.order_id)>=3 then 'Yes' else 'No' end AS Loyal
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
    WHERE oi.created_at>="2022-01-01" and oi.created_at<="2022-06-30"
    GROUP BY u.traffic_source, oi.user_id
''').to_dataframe()
connection

,Traffic_Sources,Loyal
0,Email,No
1,Email,No
2,Email,No
3,Email,No
4,Email,No
...,...,...
30663,Facebook,No
30664,Facebook,No
30665,Facebook,No
30666,Facebook,No


In [358]:
chi_df=connection

In [359]:
contigency_table = pd.crosstab(chi_df['Traffic_Sources'],chi_df['Loyal'])
contigency_table

Loyal,No,Yes
Traffic_Sources,,
Display,984,223
Email,1242,263
Facebook,1532,339
Organic,3779,865
Search,17235,4206


**H0: Traffic Sources  tidak berhubungan dengan Loyal atau Tidak Loyal**<br>
**H1: Traffic Sources berhubungan dengan Loyal atau Tidak Loyal**

In [361]:
stat, p, dof, expected = stats.chi2_contingency(contigency_table)
print('p=%.3f' % (p))
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

p=0.089
Probably independent


Dari hasil perhitungan dengan Chi-Square test antara 2 variabel yaitu Traffic Source dan Loyal menunjukkan bahwa hasil dari perhitungan Chi-Square mendapatkan hasil P = 0.089 yang mana hasil itu lebih besar dari batas nilai 0.05 atau (0.089 > 0.05), bisa juga disebut bahwa kedua variabel adalah independent. Hasilnya hipotesis menyatakan H0(Hipotesis Null) gagal di tolak dan H1(Hipotesis Alternatif) berhasil di tolak. Dapat disimpulkan bahwa kedua variabel tidak saling berhubungan.

# Poin Analisis

Setelah kita melakukan analisis secara menyeluruh pada DataSet The Look Ecommerce selama periode Januari - Juni 2022 dari total keseluruhan user yang ada, kita dapat menarik benang merah dari keseluruhan analisis.

Bisa disimpulkan user loyal sangatlah sedikit dari keseluruhan total user yang ada, hanya sekitar 19 persen user yang melakukan transaksi lebih dari 3 kali di platform The look Ecommerce. Hal ini bisa ditinjau lagi, apakah sedikitnya jumlah user yang melakukan transaksi disebabkan oleh harga yang terlalu mahal dibanding platform lain atau bisa disebabkan barang yang di jual kurang menarik untuk pembeli di negara tertentu.

Jika kita melihat dari jumlah transaksi user loyal, transaksi paling banyak yang di lakukan user loyal sebanyak 11 kali transaksi dan rata-rata kebanyakan user loyal melakukan transaksi di platform The Look Ecommerce sebanyak 3.79 kali.

Kita juga mengetahui jumlah uang yang sudah dihabiskan oleh user loyal di platform The Look Ecommerce. User loyal paling sedikit menghabiskan uang sejumlah 24.21 Dollar dan user loyal yang yang paling banyak menghabiskan uang sejumlah 1380.72 Dollar. Rata- rata uang yang dihabiskan dari seluruh user loyal di kisaran 218.64 Dollar.Tentu kita mengabaikan transaksi yang berstatus Cancelled dan Returned karena uang user tentu harus dikembalikan dan tidak masuk ke kas perusahaan.

Setelah melihat Traffic Source yang banyak digunakan oleh user loyal, Saya dapat menyarankan kepada Divisi Marketing atau Digital Marketing bisa memaksimalkan pemasaran melalui 2 tipe Traffic Source yaitu Search yang diakses oleh user sebanyak 4206 kali dan Organics yang di akses user sebanyak 865 kali. Sehingga cost iklan bisa digunakan lebih efisien dengan hasil iklan yang maksimal atau bahkan cost iklan bisa lebih murah.

Kita juga telah mengklasifikasikan user loyal minimal telah melakukan transaksi minimal 3 kali dan untuk transaksi yang masih di bawah 3 kali kita bisa bilang user tidak loyal. Perhitungan transaksi diambil dari keseluruhan transaksi yang telah dilakukan user dari Traffic Source manapun. Sehingga bisa dibilang Traffic Sources tidak mempengaruhi kategori dari user loyal maupun tidak loyal.




# Poin Pertanyaan

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!<br>
Jawab = Iya, Karena berdasarkan kaidah 5W + 1H.
2. Apa yang dimaksud dengan customer/user retention?<br>
Jawab = user retention adalah user yang telah melakukan minimal 3 kali transaksi dan dalam kasus analisis di atas user retention termasuk user loyal.

3. Jika platform The Look membuat fitur tambahan, apakah kamu akan merekomendasikan/memprioritaskan kepada user loyal? apa pertimbanganmu?<br>
Jawab = Iya, saya akan merekomendasikan kepada user loyal. Jika ditinjau dari banyaknya transaksi yang dilakukan user loyal berarti produk yang kita jual merupakan produk yang sering dibutuhkan oleh user loyal atau kita bisa sebut mereka adalah target market kita. Tentu lebih tinggi kemungkinan user loyal akan melakukan repeat order, sehingga ketika kita menawarkan fitur baru kepada user loyal, mereka cenderung akan tetap setia untuk menggunakan platform The Look karena terdapat update fitur yang mempermudah mereka.
